Universidade Federal de Alagoas

Instituto de Computação


Disciplina: Sistemas Distribuídos

Semestre letivo: 2017.2

Professor: André Lage Freitas


# Trabalho Prático AB2


## Especificação

_Importante_. Leia atentamente essa seção pois só serão aceitos trabalhos que obedecerem as a especificações a seguir. 

**Implementar o detector de falha não confiável** (_unreliable failure detector_) descrito na Seção _15.1.1 Failure assumptions and failure detectors_ do livro a seguir:

Coulouris, Dollimore, Kindberg and Blair, _Distributed Systems: Concepts and Design_. Addison-Wesley, 2012.

Os **_workers_ devem enviar ao _master_ a cada segundo o estado dos workers** que eles monitoram, indicando quais _workers_ estão suspeitos ou não suspeitos.

### Testando o trabalho

Exemplo de como testar o detector de falhas em sistemas UNIX:

```bash
kill -9 $(ps aux | grep -i julia | grep worker | awk '{print $2}' | head -1 )
```


### Algumas funções úteis

A seguir, segue uma lista (não extensiva) de funções que poderão ajudar na implementação:

```julia
addprocs
remotecall
@fetchfrom
@everywhere
sleep
@spawn
```

Para buscar uma rápida ajuda sobre as funções, utilize `?` na frente da função (ver exemplo abaixo). 

```julia
?addprocs
```

Utilize a [documentação oficial da linguagem Julia (0.6.2)](https://docs.julialang.org/en/stable/) para demais dúvidas.

## Código-fonte

Inicia 4 processos

In [1]:
addprocs(4)

4-element Array{Int64,1}:
 2
 3
 4
 5

Funções dos processos

In [2]:
@everywhere suspects = Int[]
@everywhere receive_alive = Int[]
@everywhere workers_ids = Int[]
@everywhere my_id = Int

In [3]:
@everywhere function send_alive()
    global workers_ids
    global my_id
    for id in workers_ids
        if id != my_id
            try
                fetch(remotecall(alive, id, my_id))
            catch
                global suspects
                if length(findin(suspects, id)) == 0
                    push!(suspects, id)
                end
            end
        end
    end
end

@everywhere function alive(id)
    global receive_alive
    global suspects
    if length(findin(receive_alive, id)) == 0
        push!(receive_alive, id)
    end
    deleteat!(suspects, findin(suspects, id))
end

#=
@everywhere function get_suspects()
    global suspects
    return suspects
end
=#

@everywhere function init(all_ids, worker_id)
    global my_id = worker_id
    global workers_ids = all_ids
end

@everywhere function check_suspects()
    global my_id
    global workers_ids
    global receive_alive
    global suspects
    
    for id in receive_alive
        deleteat!(suspects, findin(suspects, id))
    end
    for id in workers_ids
        if id != my_id && length(findin(receive_alive, id)) == 0 && length(findin(suspects, id)) == 0
            if length(findin(suspects, id)) == 0
                push!(suspects, id)
            end
        end
    end
    receive_alive = Int[]
end

@everywhere function main_loop()
    global suspects
    global my_id
    try
        while true
            ntimes = 20
            for i in 1:(ntimes)
                try
                    fetch(remotecall(send_alive, my_id))
                end
                sleep(1.0/(ntimes))
            end
            
            #sleep(2)

            try
                fetch(remotecall(check_suspects, my_id))
                if length(suspects) > 0
                    println("suspects: $suspects")
                end
            end
        end
    end
end

In [4]:
for id in workers()
    try
        fetch(remotecall(init, id, workers(), id))
    catch
        println("error to initialize worker number $id")
    end
end

#s = nothing
for id in workers()
    try
        #=if s == nothing
            s = remotecall(main_loop, id)
        else
            s = remotecall(main_loop, id) .+ s
        end=#
        @async fetch(remotecall(main_loop, id))
    end
end

#fetch(s)

#=
while true
    if length(workers()) == 0
        break
    end
    
    sleep(2)
    
    for id in workers()
        try
            fetch(remotecall(send_alive, id))
        catch
            continue
        end
    end
    
    for id in workers()
        try
            fetch(remotecall(check_suspects, id))
            s = fetch(remotecall(get_suspects, id))
            
            if length(s) > 0
                println("suspects for worker $id: $s")
            end
        catch
            continue
        end
    end
end
=#

Worker 2 terminated.ERROR (unhandled task failure): ProcessExitedException()
Stacktrace:
 [1] try_yieldto(::Base.##296#297{Task}, ::Task) at ./event.jl:189
 [2] wait() at ./event.jl:234
 [3] wait(::Condition) at ./event.jl:27
 [4] wait_impl(::Channel{Any}) at ./channels.jl:364
 [5] wait(::Channel{Any}) at ./channels.jl:360
 [6] take_buffered at ./channels.jl:319 [inlined]
 [7] take!(::Channel{Any}) at ./channels.jl:317
 [8] #remotecall_fetch#141(::Array{Any,1}, ::Function, ::Function, ::Base.Distributed.Worker, ::Base.Distributed.RRID, ::Vararg{Any,N} where N) at ./distributed/remotecall.jl:350
 [9] remotecall_fetch(::Function, ::Base.Distributed.Worker, ::Base.Distributed.RRID, ::Vararg{Any,N} where N) at ./distributed/remotecall.jl:346
 [10] #remotecall_fetch#144(::Array{Any,1}, ::Function, ::Function, ::Int64, ::Base.Distributed.RRID, ::Vararg{Any,N} where N) at ./distributed/remotecall.jl:367
 [11] call_on_owner(::Function, ::Future) at ./distributed/remotecall.jl:440
 [12] fetch(:

	From worker 4:	suspects: [2]
	From worker 5:	suspects: [2]
	From worker 3:	suspects: [2]
	From worker 3:	suspects: [2]
	From worker 4:	suspects: [2]
	From worker 5:	suspects: [2]
	From worker 5:	suspects: [2]
	From worker 4:	suspects: [2]
	From worker 3:	suspects: [2]
	From worker 4:	suspects: [2]
	From worker 3:	suspects: [2]
	From worker 5:	suspects: [2]
	From worker 3:	suspects: [2]
	From worker 5:	suspects: [2]
	From worker 4:	suspects: [2]
	From worker 4:	suspects: [2]
	From worker 3:	suspects: [2]
	From worker 5:	suspects: [2]


Worker 3 terminated.ERROR (unhandled task failure): ProcessExitedException()
Stacktrace:
 [1] try_yieldto(::Base.##296#297{Task}, ::Task) at ./event.jl:189
 [2] wait() at ./event.jl:234
 [3] wait(::Condition) at ./event.jl:27
 [4] wait_impl(::Channel{Any}) at ./channels.jl:364
 [5] wait(::Channel{Any}) at ./channels.jl:360
 [6] take_buffered at ./channels.jl:319 [inlined]
 [7] take!(::Channel{Any}) at ./channels.jl:317
 [8] #remotecall_fetch#141(::Array{Any,1}, ::Function, ::Function, ::Base.Distributed.Worker, ::Base.Distributed.RRID, ::Vararg{Any,N} where N) at ./distributed/remotecall.jl:350
 [9] remotecall_fetch(::Function, ::Base.Distributed.Worker, ::Base.Distributed.RRID, ::Vararg{Any,N} where N) at ./distributed/remotecall.jl:346
 [10] #remotecall_fetch#144(::Array{Any,1}, ::Function, ::Function, ::Int64, ::Base.Distributed.RRID, ::Vararg{Any,N} where N) at ./distributed/remotecall.jl:367
 [11] call_on_owner(::Function, ::Future) at ./distributed/remotecall.jl:440
 [12] fetch(:

	From worker 4:	suspects: [2]
	From worker 5:	suspects: [2]
	From worker 5:	suspects: [2, 3]
	From worker 4:	suspects: [2, 3]
	From worker 4:	suspects: [2, 3]
	From worker 5:	suspects: [2, 3]
	From worker 5:	suspects: [2, 3]
	From worker 4:	suspects: [2, 3]
	From worker 4:	suspects: [2, 3]
	From worker 5:	suspects: [2, 3]
	From worker 4:	suspects: [2, 3]
	From worker 5:	suspects: [2, 3]
	From worker 4:	suspects: [2, 3]
	From worker 5:	suspects: [2, 3]
	From worker 5:	suspects: [2, 3]
	From worker 4:	suspects: [2, 3]
	From worker 5:	suspects: [2, 3]
	From worker 4:	suspects: [2, 3]
	From worker 5:	suspects: [2, 3]
	From worker 4:	suspects: [2, 3]
	From worker 4:	suspects: [2, 3]
	From worker 5:	suspects: [2, 3]
	From worker 5:	suspects: [2, 3]
	From worker 4:	suspects: [2, 3]
	From worker 4:	suspects: [2, 3]
	From worker 5:	suspects: [2, 3]
	From worker 5:	suspects: [2, 3]
	From worker 4:	suspects: [2, 3]
	From worker 5:	suspects: [2, 3]
	From worker 4:	suspects: [2, 3]
	From worker 5:	


## Entrega

As respostas deverão ser entregues na parte indicada desse arquivo reservada ao código fonte, no formato [IJulia Notebook](https://github.com/JuliaLang/IJulia.jl), que utiliza tecnologia [Jupyter](https://www.jupyter.org). Utilize explicações sobre o código em [Markdown](https://en.wikipedia.org/wiki/Markdown).

O programa deve ser implementado na linguagem de programação [**Julia** versão 0.6.2](https://docs.julialang.org/en/stable/). Seu trabalho será testado na [JuliaBox](https://juliabox.com), portanto teste seu arquivo `.ipynb` IJulia na JuliaBox antes de enviá-lo.


### Forma 

O arquivo IJulia Notebook deverá ser entregue ao Professor **exclusivamente através do Google Classroom**.



A responsabilidade sobre a integridade do arquivo contendo trabalho é exclusivamente dos discentes. Serão ignorados os trabalhos cujos arquivos não conseguirem ser abertos pelo Professor.

### Prazos

O prazo de entrega está descrito no **Google Classroom**.


## Pontuação extra

O(a) discente que realizar mais tarefas, além do que foi especificado neste trabalho, o professor atribuirá de 0,5 a 1,0 ponto extra a depender da relevância da contribuição no programa. O critério será decidido pelo professor. 

O(a) discente deve indicar qual é a tarefa executada. Por exemplo, adição de funcionalidades, armazenamento de dados em arquivo, documentação de código, comentários em inglês sem erros ortográficos, etc.


**Plágio** A nota zero será atribuída caso haja qualquer tipo de cópia parcial ou integral assim como as devidas medidas legais. Leia a [cartilha sobre plágio](http://www.noticias.uff.br/arquivos/cartilha-sobre-plagio-academico.pdf).

## Disclaimer

Esse material foi elaborado pelo [Prof. André Lage Freitas](https://sites.google.com/a/ic.ufal.br/andrelage/) e está licenciado sob a licença _GNU General Public License v3.0_.